In [ ]:
#!pip install transformers

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
import torch
from transformers import BertTokenizer
from transformers import BertForQuestionAnswering
#from question_answering import load_pretrained_qa_model
#from question_answering import answer_question

In [2]:
#path information
task='task1'# or 'task2' # specify task
#root_path='/repo1/code/autoreview/'
root_path='./'
data_path=root_path+'data/'+task+'/'
save_path=root_path+'results/'+task+'/'
submission_path='./'
ranking_file_name='pseudo_label.pkl' #'ranking.tsv'
question_file_name='questions_structured.csv'
literature_file_name='metadata_hypercoagulable.tsv'
answer_confidence_threshold=1
top_k=300 # top_k articles
col='abstract'

In [3]:
def load_pretrained_qa_model(model_str=None, use_cuda=True):
    if model_str is None:
        model_str = 'bert-large-uncased-whole-word-masking-finetuned-squad'
        device = torch.device('cuda' if torch.cuda.is_available() and use_cuda else 'cpu')
    
    tokenizer = BertTokenizer.from_pretrained(model_str)
    model = BertForQuestionAnswering.from_pretrained(model_str).to(device)

    model.eval()
    return tokenizer, model

def answer_question(question, document, model, tokenizer):
    device = model.device
    
    encoded = tokenizer.encode_plus(question, document, return_tensors='pt', max_length=512)
    start_scores, end_scores = model(encoded['input_ids'].to(device),
                                     token_type_ids=encoded['token_type_ids'].to(device))

    tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'].squeeze())
    ans_start, ans_end = torch.argmax(start_scores), torch.argmax(end_scores)
    
    ans_tokens = tokens[ans_start: ans_end+1]
    if '[SEP]' in ans_tokens:
        ans_tokens = ans_tokens[ans_tokens.index('[SEP]')+1:]
    ans = tokenizer.convert_tokens_to_string(ans_tokens)
    ans = ans.replace(' - ', '-').replace('[CLS]', '')
    ans_score = start_scores.max() + end_scores.max()

    return ans, ans_score.item()

def ask_all_questions(abstract, ncord_uid):
    answers = []
    for question in questions['question'].values:
        ans, score= answer_question(question, abstract, model=model, tokenizer=tokenizer)
        if ans !='':
            answers.append((ncord_uid, question, ans, score))
    if len(answers) == 0:
        return None
    return answers

Load QA models

In [5]:
tokenizer, model = load_pretrained_qa_model()

Prepare ranked articles

In [4]:
ranking=pickle.load(open(save_path+ranking_file_name, 'rb'))
#ranking=ranking.drop(columns=['Unnamed: 0'],axis=1).reset_index(drop=True)
ranking=ranking.reset_index(drop=True)
ranking=ranking.loc[ranking['label']==1]
ranking_top_k=ranking.iloc[:top_k,:]

Ask questions and get answers

In [6]:
questions=pd.read_csv(data_path+question_file_name, header=None, names=['type','question'], )

In [7]:
_answers_all=ranking_top_k.apply(lambda row: ask_all_questions(row[col], row['ncord_uid']),axis=1)


In [8]:
answers_all = [item for list in _answers_all.dropna().values for item in list]
answers_all=pd.DataFrame(answers_all, columns=['ncord_uid', 'question', 'answer', 'score'])
answers_all=answers_all.loc[answers_all['score']>answer_confidence_threshold]
answers_all_pivot=answers_all.pivot(index='ncord_uid',columns='question', values='answer')

In [9]:
index_category_question=pd.MultiIndex.from_frame(questions, names=['category','question'])
answers_all_pivot.set_axis(index_category_question, axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: set_axis currently defaults to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  


Load article's meta file

In [10]:
literature=pd.read_csv(data_path+literature_file_name, sep='\t', index_col='ncord_uid').drop(columns='Unnamed: 0')

In [11]:
metadata=['published','journal', 'title', 'abstract','cord_uid']
idx=[('meta', col) for col in metadata]
idx=pd.MultiIndex.from_tuples(idx)
literature_multiindex=literature[metadata]
literature_multiindex.set_axis(idx, axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: set_axis currently defaults to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  """


Merge the meta information with extracted information

In [12]:
summary=pd.merge(answers_all_pivot, literature_multiindex, how='left',left_index=True, right_index=True )

In [15]:
summary.to_csv(save_path+'summary.csv')

In [13]:
summary

category                                          study type  \
question                     What was the type of the study?   
ncord_uid                                                      
1                                                        NaN   
94         objective to provide an update to the " surviv...   
259                                                      NaN   
297                                                      NaN   
314                                                      NaN   
...                                                      ...   
7600                                                     NaN   
7602                                                     199   
7655                                                     NaN   
7713                                                     NaN   
7743                                                     NaN   

category                                                \
question  Was it a Prospective, Retrospective Studies?   
ncord_uid                                                
1                                                  NaN   
94                                                 NaN   
259                                                NaN   
297                                                NaN   
314                                                NaN   
...                                                ...   
7600                                               NaN   
7602                                               NaN   
7655                                               NaN   
7713                                               NaN   
7743                                               NaN   

category                                         \
question  Was it a Randomized Controlled Trial?   
ncord_uid                                         
1                                           NaN   
94                                          NaN   
259                                         NaN   
297                                         NaN   
314                                         NaN   
...                                         ...   
7600                                        NaN   
7602                                        NaN   
7655                                        NaN   
7713                                        NaN   
7743                                        NaN   

category                                          \
question  Was it a case control or Cohort Study?   
ncord_uid                                          
1                                            NaN   
94                                           NaN   
259                                          NaN   
297                                          NaN   
314                                    fifty-two   
...                                          ...   
7600                                         117   
7602                                         NaN   
7655                                         NaN   
7713                                        2162   
7743                                          39   

category                                  Therapeutic method  \
question               What therapeutic method was utilized?   
ncord_uid                                                      
1                                                          .   
94                                                       NaN   
259                                                      NaN   
297               this was a retrospective multicenter study   
314                          prospective observational study   
...                                                      ...   
7600       a prospective , multicentre series was conduct...   
7602                                                     NaN   
7655             prospective parallel-group randomized study   
7713       prospective assessment of dual antiplatelet th...   
7743       pur